In [46]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Input, Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot as plt

In [47]:


model1 = Sequential([
    Input(shape=(4,)),
    Dense(50, 'tanh'),
    Dense(50, 'tanh'),
    Dense(50, 'tanh'),
    Dense(100, 'tanh'),
    Dense(100, 'tanh'),
    Dense(100, 'tanh'),
    Dense(100, 'tanh'),
    Dense(2, 'tanh', name='Output')
], name='u(x, y),v(x,y)')

model1.summary()

Model: "u(x, y),v(x,y)"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 50)                250       
                                                                 
 dense_13 (Dense)            (None, 50)                2550      
                                                                 
 dense_14 (Dense)            (None, 50)                2550      
                                                                 
 dense_15 (Dense)            (None, 100)               5100      
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dense_17 (Dense)            (None, 100)               10100     
                                                                 
 dense_18 (Dense)            (None, 100)            

In [48]:
num_points =50
x = np.linspace(-1, 1, num_points)
y = np.linspace(-1, 1, num_points)
fx = np.linspace(0, 2, num_points)
fy = np.linspace(0, 2, num_points)

grid_x, grid_y,grid_fx,grid_fy = np.meshgrid(x, y,fx,fy)
states = np.array(list(zip(grid_x.reshape(-1), grid_y.reshape(-1),grid_fx.reshape(-1),grid_fy.reshape(-1))))

In [49]:
G = tf.cast(tf.constant(1), tf.float32)
E = tf.cast(tf.constant(2.6), tf.float32)
v = tf.cast(tf.constant(0.3), tf.float32)
Lambda = tf.cast(tf.constant(.857142857), tf.float32)


#v=0.3,G=1,E=2.6

def differential_loss(model, training_dat):

    training_data = tf.Variable(tf.cast(training_dat, tf.float32))

    tds = training_data.shape[0]




    with tf.GradientTape(persistent=True) as tape_dubgrad:
       with tf.GradientTape(persistent=True) as tape_grad:
              outputs = model(training_data)

              du_dx__du_dy__du_dkx__du_dky = tape_grad.gradient(outputs[:,0], training_data)
              dv_dx__dv_dy__dv_dkx__dv_dky = tape_grad.gradient(outputs[:,1], training_data)
              D_Du = du_dx__du_dy__du_dkx__du_dky
              D_Dv = dv_dx__dv_dy__dv_dkx__dv_dky
              du_dx,du_dy,du_dkx,du_dky= D_Du[:,0],D_Du[:,1],D_Du[:,2],D_Du[:,3]
              dv_dx,dv_dy,dv_dkx,dv_dky= D_Dv[:,0],D_Dv[:,1],D_Dv[:,2],D_Dv[:,3]

              d2u_dy2 = tape_dubgrad.gradient(du_dy, training_data)[:,1]
              d2v_dy2 = tape_dubgrad.gradient(dv_dy,training_data)[:,1]

              d2u_dx2 = tape_dubgrad.gradient(du_dx, training_data)[:,0]
              d2v_dx2 = tape_dubgrad.gradient(dv_dx, training_data)[:,0]


              d2v_dxdy = tape_dubgrad.gradient(dv_dy, training_data)[:,0]
              del tape_grad
       d2u_dxdy = tape_dubgrad.gradient(du_dy, training_data)[:,0]
       d2u_dydx = tape_dubgrad.gradient(du_dx, training_data)[:,1]


      #  dux_vy_dx = tape_dubgrad.gradient(du_dx + dv_dy,training_data[:,0])
      #  dux_vy_dy = tape_dubgrad.gradient(du_dx + dv_dy,training_data[:,1])
       dux_vy_dx = d2u_dx2 + d2v_dxdy
       dux_vy_dy = d2u_dydx + d2v_dy2
    del tape_dubgrad



    diff_loss = (G*(d2u_dx2 + d2u_dy2) + (G+Lambda)*(dux_vy_dx) + training_data[:,2])**2 + (G*(d2v_dx2 + d2v_dy2) + (G+Lambda)*(dux_vy_dy) + training_data[:,3])**2
    diff_loss = tf.reduce_sum(diff_loss)
    return diff_loss


def boundary_loss(model,training_dat):


    training_data = tf.Variable(tf.cast(training_dat, tf.float32))


    states_x_1 = training_data[training_data[:, 0] == -1]
    states_y_1 = training_data[training_data[:, 1] == -1]
    states_kx0ky0 = training_data[training_data[:, 2]**2 + training_data[:, 3]**2 == 0]

    tds = training_data.shape[0]
    states_y_1 = tf.Variable(states_y_1)



    bound_loss = (model(states_x_1)[:,0])**2 + (model(states_x_1)[:,1])**2


    bound_loss = tf.reduce_sum(bound_loss)+ tf.reduce_sum((model(states_kx0ky0)[:,0])**2) + tf.reduce_sum((model(states_kx0ky0)[:,1])**2)
    return bound_loss

In [50]:
def compute_cost(model, training_data, _lambda=0):
    return 100*boundary_loss(model,training_data) + differential_loss(model, training_data)

In [51]:
# print(compute_cost(model1,states))

In [52]:
# print(compute_cost(model1,states))

In [53]:
def train_model(model, states, num_epochs=300, lr=0.00001, decay=0.000001, _lambda=0):
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr, weight_decay=decay)
    cost_history = []
    # initial_lr = 0.00009  # Initial learning rate
    # decay_steps = 1000  # Number of steps before decaying the learning rate
    # decay_rate = 0.5  # Decay rate for the learning rate

    # learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay( initial_learning_rate=initial_lr,decay_steps=decay_steps,decay_rate=decay_rate)

    # optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)

    for epoch in range(num_epochs):
        with tf.GradientTape() as tape:
            cost_value = compute_cost(model, states, _lambda=_lambda)

        gradients = tape.gradient(cost_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        cost_history.append(cost_value)

        print("epochs",epoch)
        print(cost_value)
        if(cost_value<1e-5):
          break

    return model, cost_history


In [54]:
_, cost_history = train_model(model1,states, num_epochs=2000, lr=0.0001, decay=0.00001, _lambda=0)
plt.plot(range(len(cost_history)), cost_history)
plt.xlabel('Number of iterations')
plt.ylabel('Cost')
plt.title('Behaviour with epochs')

plt.show()

ResourceExhaustedError: ignored

In [ ]:
magnitude_fx = 0
magnitude_fy = 1
x = np.linspace(-1, 1, num_points)
y = np.linspace(-1, 1, num_points)
fx = np.linspace(magnitude_fx,magnitude_fx, num_points)
fy = np.linspace(magnitude_fy, magnitude_fy, num_points)

grid_x, grid_y,grid_fx,grid_fy = np.meshgrid(x, y,fx,fy)
new_states = np.array(list(zip(grid_x.reshape(-1), grid_y.reshape(-1),grid_fx.reshape(-1),grid_fy.reshape(-1))))

plt.scatter(new_states[:,0], new_states[:,1])
UV = model1(new_states)
plt.scatter(new_states[:,0]+UV[:,0] , new_states[:,1] + UV[:,1], alpha=0.1)

In [30]:
model1.save_weights("pinn1.h5")